# <center> *Big Data project* </center> 
### <center> *Bald model* </center>
_<center>M2 Statistics & Econometrics, 2019-2020</center>_



In [0]:
import os
import zipfile
import random
import pickle
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import expand_dims
from PIL import Image
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip uninstall tensorflow -gpu -y
!pip install tensorflow-gpu==2.1.0
import tensorflow as tf
print(tf.__version__)
import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import AveragePooling2D, MaxPool2D, Conv2D, MaxPooling2D, InputLayer 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from keras.utils import np_utils, to_categorical
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import load_model


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -g
2.1.0


Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with zipfile.ZipFile('/content/drive/My Drive/Big Data/Projet/celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/img/')

In [0]:
path = '/content/img/img_align_celeba/img_align_celeba/'
samp = pickle.load(open('/content/drive/My Drive/Big Data/Projet/Listes sample/Bald.pkl', 'rb'))

In [0]:
target = []
data = np.zeros((len(samp), 64, 64, 3)) #dimension of the images
count = 0
for file in os.listdir(path) :
  if file in samp :
    image = Image.open(path + file)
    image = image.resize((64, 64))
    data[count] = np.array(image)
    target.append(file)
    if count%1000 == 0 :
      print(count)
    count += 1
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [0]:
df_attr = pd.read_csv('/content/img/list_attr_celeba.csv')
df_attr.replace(to_replace = -1, value = 0, inplace = True)
bald = df_attr[['image_id', 'Bald']].set_index('image_id')
bald = bald.loc[target]
print(bald.shape)
bald.head()

(11530, 1)


,Bald
image_id,
193444.jpg,0
145586.jpg,0
095195.jpg,0
201969.jpg,1
021385.jpg,0


### **Baseline model**

In [0]:
target = to_categorical(bald.Bald)
print(target.shape)
print(data.shape)

(11530, 2)
(11530, 64, 64, 3)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size = 0.2, random_state = 0)
print("Train set dimension: ", X_train.shape, y_train.shape)
print("Test set dimension: ", X_test.shape, y_test.shape)

Train set dimension:  (9224, 64, 64, 3) (9224, 2)
Test set dimension:  (2306, 64, 64, 3) (2306, 2)


In [0]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(optimizers.Adam(lr = 0.001), loss = "categorical_crossentropy", 
              metrics = ["accuracy"])

In [0]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10, 
          validation_data = (X_test, y_test))

Train on 9224 samples, validate on 2306 samples
Epoch 1/10
9224/9224 [==============================] - 6s 604us/sample - loss: 4.9850 - accuracy: 0.7960 - val_loss: 0.3630 - val_accuracy: 0.8539
Epoch 2/10
9224/9224 [==============================] - 2s 237us/sample - loss: 0.3349 - accuracy: 0.8619 - val_loss: 0.3447 - val_accuracy: 0.8578
Epoch 3/10
9224/9224 [==============================] - 2s 236us/sample - loss: 0.2790 - accuracy: 0.8844 - val_loss: 0.3553 - val_accuracy: 0.8391
Epoch 4/10
9224/9224 [==============================] - 2s 234us/sample - loss: 0.2529 - accuracy: 0.8939 - val_loss: 0.3549 - val_accuracy: 0.8643
Epoch 5/10
9224/9224 [==============================] - 2s 234us/sample - loss: 0.2023 - accuracy: 0.9204 - val_loss: 0.4228 - val_accuracy: 0.8504
Epoch 6/10
9224/9224 [==============================] - 2s 235us/sample - loss: 0.1937 - accuracy: 0.9238 - val_loss: 0.3296 - val_accuracy: 0.8643
Epoch 7/10
9224/9224 [==============================] - 2s 236us

### **Model**

In [0]:
target = bald.Bald
print(target.shape)
print(data.shape)

(11530,)
(11530, 64, 64, 3)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, 
                                                    random_state = 0)
print("Train set dimension: ", X_train.shape, y_train.shape)
print("Test set dimension: ", X_test.shape, y_test.shape)

Train set dimension:  (9224, 64, 64, 3) (9224,)
Test set dimension:  (2306, 64, 64, 3) (2306,)


In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizers.Adam(lr = 0.001), loss = "binary_crossentropy", 
              metrics = ["accuracy"])

In [0]:
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 1, 
                              min_lr = 0.001)

save_models = "/content/drive/My Drive/Big Data/Projet/Models/"
model_name = "Best_Bald_model.hdf5"
file_path = save_models + model_name
checkpoint = ModelCheckpoint(file_path, monitor = 'val_accuracy', verbose = 1,
                             save_best_only = True, mode = 'max')

callbacks = [checkpoint, reduce_lr]

In [0]:
model.fit(X_train, y_train, batch_size = 32, epochs = 20, 
          validation_data = (X_test, y_test), callbacks = callbacks)

Train on 9224 samples, validate on 2306 samples
Epoch 1/20
9152/9224 [============================>.] - ETA: 0s - loss: 0.6871 - accuracy: 0.7681
Epoch 00001: val_accuracy improved from -inf to 0.86947, saving model to /content/drive/My Drive/Big Data/Projet/Models/Best_Bald_model.hdf5
9224/9224 [==============================] - 4s 445us/sample - loss: 0.6851 - accuracy: 0.7679 - val_loss: 0.3419 - val_accuracy: 0.8695
Epoch 2/20
9120/9224 [============================>.] - ETA: 0s - loss: 0.3302 - accuracy: 0.8612
Epoch 00002: val_accuracy did not improve from 0.86947
9224/9224 [==============================] - 2s 241us/sample - loss: 0.3289 - accuracy: 0.8620 - val_loss: 0.3186 - val_accuracy: 0.8630
Epoch 3/20
9088/9224 [============================>.] - ETA: 0s - loss: 0.2731 - accuracy: 0.8894
Epoch 00003: val_accuracy improved from 0.86947 to 0.88465, saving model to /content/drive/My Drive/Big Data/Projet/Models/Best_Bald_model.hdf5
9224/9224 [==============================] -